In [2]:
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error

In [3]:
# Going to use Tesla as the stock I work with

ticker = "TSLA"
start_date = '2017-11-30'
end_date = '2020-11-30'

In [ ]:
stock_date = data.DataReader(ticker, 'iex', start_date, end_date)
print(stock_date.shape)
stock_date.head()

In [2]:
# Try using another aggregation api.

In [1]:
# Need to build out the RSI for this project as well.

In [7]:
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error
import yahoo_finance as yf

In [9]:
m = Prophet(weekly_seasonality=False,yearly_seasonality=False)
m.add_seasonality('self_define_cycle',period=8,fourier_order=8,mode='additive')

In [1]:
#import the libraries
import math
import warnings
import datetime 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from pandas_datareader import DataReader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Dense, LSTM

warnings.filterwarnings("ignore")

In [2]:
#get the stock quote for the past 10 years
stock = input("Enter a stock ticker: ")
num_of_years = 10
start_date = datetime.datetime.now() - datetime.timedelta(days=int(365.25*num_of_years))
end_date = datetime.date.today()

Enter a stock ticker: AAPL


In [3]:
df = DataReader(stock, "yahoo", start_date, end_date)
data = df.filter(['Close'])
dataset = data.values
train_data_len = math.ceil(len(dataset)*.8)

In [4]:
df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-12-27,11.622857,11.482857,11.530357,11.595715,249816000.0,10.003056
2010-12-28,11.666429,11.609285,11.639643,11.623929,175924000.0,10.027392
2010-12-29,11.658929,11.610714,11.650714,11.617500,163139200.0,10.021846
2010-12-30,11.625357,11.537500,11.624286,11.559286,157494400.0,9.971630
2010-12-31,11.552857,11.475357,11.533929,11.520000,193508000.0,9.937736
...,...,...,...,...,...,...
2020-12-17,129.580002,128.039993,128.899994,128.699997,94359800.0,128.699997
2020-12-18,129.100006,126.120003,128.960007,126.660004,192541500.0,126.660004
2020-12-21,128.309998,123.449997,125.019997,128.229996,121251600.0,128.229996


In [8]:
#scale the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

#create the training dataset
train_data = scaled_data[0:train_data_len, :]

#split the data into x_train and y_train dataset
x_train=[]
y_train=[]

for i in range(60,len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i<=61:
        #print(x_train)
        #print(y_train)
        print()

#convert x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

#reshape the data to 3 dimension
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [10]:
#build LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(50,return_sequences=False))

model.add(Dense(25))
model.add(Dense(1))

#compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

#train the model
model.fit(x_train, y_train, batch_size=1, epochs=5)

#create test dataset
test_data = scaled_data[train_data_len-60:, :]

#create dataset x_test, y_test
x_test = []
y_test = dataset[train_data_len:, :]
for i in range(60,len(test_data)):
    x_test.append(test_data[i-60:i, 0])

#convert data to numpy array
x_test = np.array(x_test)

#reshape the data
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

#get the models predicted price values
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

Epoch 1/5
1954/1954 [==============================] - 27s 14ms/step - loss: 2.4875e-04 - accuracy: 5.1177e-04
Epoch 2/5
1954/1954 [==============================] - 29s 15ms/step - loss: 1.1464e-04 - accuracy: 5.1177e-04
Epoch 3/5
1954/1954 [==============================] - 30s 15ms/step - loss: 9.4276e-05 - accuracy: 5.1177e-04
Epoch 4/5
1954/1954 [==============================] - 28s 15ms/step - loss: 7.2183e-05 - accuracy: 5.1177e-04
Epoch 5/5
1954/1954 [==============================] - 30s 15ms/step - loss: 6.1708e-05 - accuracy: 5.1177e-04
